In [ ]:
import time
import json
import ast
import os
import datetime
import io
from collections import defaultdict

import imageio
import boto3
import pandas as pd
import numpy as np
import imageio
import matplotlib.pyplot as plt
import seaborn as sns

from brtdevkit.core.db.athena import AthenaClient
from brtdevkit.data import Dataset
from timezonefinder import TimezoneFinderL
import pytz

from aletheia_dataset_creator.dataset_tools.aletheia_dataset_helpers import imageids_to_dataset
from aletheia_dataset_creator.config.dataset_config import LEFT_CAMERAS, ALL_CAMERA_PAIRS_LIST
%matplotlib inline
pd.set_option('display.max_rows', 500)

In [ ]:
athena = AthenaClient()
s3 = boto3.resource('s3')
tf = TimezoneFinderL()
home = os.path.expanduser('~')
outpath = os.environ['OUTPUT_PATH']
data_path = home + '/data'

In [ ]:
df_cache = outpath + "/skipped_images.parquet"
try:
    skip_df = pd.read_parquet(df_cache)
except FileNotFoundError:
    print("file not found")
    query = f"""
    SELECT id, hard_drive_name, robot_name, collected_on,
        bag_name, operating_field_name, operation_time, latitude, longitude, geohash, camera_location, sensor_type, created_at, 
        bundle, gps_can_data__json, weather_summary__json, group_id
    FROM image_jupiter 
    WHERE SUBSTR(camera_location, 0, 1) IN ('T', 'I')
    """
#--AND camera_location IN {left_tractor_cameras}
    start = time.time()
    skip_df = athena.get_df(query)
    end = time.time()
    print(end - start)
    skip_df.to_parquet(df_cache, index=False)